# Introduction to Remote Sensing, Remote Sensing Time Series Analysis, and Remote Sensing Indices

---

**Objectives:**

By the end of this exercise, you should be able to:
* explain what optical data is and give an example of how it is used
* explain the difference between remote sensing raster data and other raster data
* describe and calculate the Normalized Difference Water Index (NDWI)
* describe and plot the changes in NDWI over time

---

This exercise builds on our raster processes we covered in `exercise2`. In this exercise, we are focusing on a subset of raster data from satellites, known as **remotely sensed** or **optical** data.

## Exploring Planet imagery - PlanetScope

<img src="imgs/image.png" width="650">

The PlanetScope satellite constellation consists of multiple launches of groups of individual “small-satellites” (~200+ in total). Planet's on-orbit capacity is constantly improving in capability or quantity, with technology improvements deployed at a rapid pace. Each PlanetScope satellite is a CubeSat 3U form factor (10 cm by 10 cm by 30 cm). The complete PlanetScope is able to image the entire Earth every day (equating to a daily collection capacity of ~200 million km²/day).

In this in-class and homework data activity we will work on the Albemarle-Pamlico sound Peninsula/the Alligator River Basin, a coastal area in NC that is experiencing tree dieback (a.k.a. a ‘ghost forest’). 

We will be working on with surface reflectance PlanetScope OrthoTiles.

### Planet Surface reflectance

From Planet's product guide:

*"Planet’s Surface Reflectance (SR) Product is derived from the standard Analytic Product (Radiance) and is
processed to top-of-atmosphere reflectance and then atmospherically corrected to bottom-of-atmosphere
reflectance. This product ensures consistency across localized atmospheric conditions, minimizing uncertainty
in spectral response across time and location."*

*Surface Reflectance is available for all orthorectified scenes produced by radiometrically calibrated
sun-synchronous orbit Dove and RapidEye satellites. The Surface Reflectance product is available in the API as
the ‘analytic_sr’ asset under the PSScene4Band, PSOrthoTile, and REOrthoTile Itemtypes."*

### Planet Scope OrthoTiles

From Planet's website:

*"The PlanetScope Analytic Ortho Tile product is orthorectified, multispectral data from the satellite constellation. Analytic products are calibrated multispectral imagery products that have been processed to allow analysts to derive information products for data science and analytics. This product is designed for a wide variety of applications that require imagery with an accurate geolocation and cartographic projection. It has been processed to remove distortions caused by terrain and can be used for many data science and analytic applications. It eliminates the perspective effect on the ground (not on buildings), restoring the geometry of a vertical shot. The orthorectified visual imagery is optimal for value-added image processing including vegetation indices, land cover classifications, etc. In addition to orthorectification, the imagery has radiometric corrections applied to correct for any sensor artifacts and transformation to at- sensor radiance."*

From Planet's product guide:

<img src = "imgs/conversionOrtho.png" width = '400'>

### More information can be found here:

https://www.planet.com/company/resources/




## Mosaic Multiple PlanetScope orthotiles

The data that will be used for this exercise is available in the "data/folder". The folder contains suface reflectance Orthotiles from PlanetScope and RapidEye Orthotiles. Load the **SR_clip.tiff** images to QGIS, and you will realize that each image itself does not cover the entire study region, therefore, we need to merge them (i.e. create an image mosaic). 

In this exercise, we will be using a new package, `earthpy` to help us with our raster visualizations.

From EathPy repo website:

*"Python is a generic programming language designed to support many different applications. Because of this, many commonly performed spatial tasks for science including plotting and working with spatial data take many steps of code. EarthPy builds upon the functionality developed for raster data (rasterio) and vector data (geopandas) in Python"*.

#### EarthPy (more info here: https://pypi.org/project/earthpy/)


In [ ]:
# import modules
import os
from glob import glob #here is a very useful package to list files in a folder
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
import matplotlib.pyplot as plt
import earthpy.plot as ep
import earthpy.spatial as es
import numpy as np

In [ ]:
# list all PlanetScope imgs
PS_imgs = glob(os.path.join("data", "PlanetScope", '*SR_clip*.tif')) # note that we are using 2 wildcards here

# using an empty list to store the images that we will be reading with rasterio
PS_src_imgs = []

# lets loop through our list, open and append to PS_src_imgs
for img in PS_imgs:
    PS_src = rio.open(img)
    PS_src_imgs.append(PS_src)
    
# now we just need to merge the files within the source list using rasterio’s merge function
PS_mosaic, PS_out_trans = merge(PS_src_imgs)

In [ ]:
# Copy the metadata
PS_out_meta = PS_src_imgs[0].meta.copy()

# Update the metadata to save the mosaic
PS_out_meta.update({"driver": "GTiff",
                 "height": PS_mosaic.shape[1],
                 "width": PS_mosaic.shape[2],
                 "transform": PS_out_trans})
                 
# create path and nome of PS mosaic
PS_path_name = "data/PlanetScope/PS_2019-08-29_mosaic.tif"
with rio.open(PS_path_name, "w", **PS_out_meta) as dest:
     dest.write(PS_mosaic)

In [ ]:
# get PS_final mosaic source
src = rio.open("data/PlanetScope/PS_2019-08-29_mosaic.tif")

# create fig and subplots!
fig, (axb, axg, axr, axn) = plt.subplots(1,4, figsize=(20,5))
show((src, 1), ax=axb, cmap='Blues', title='blue band')
show((src, 2), ax=axg, cmap='Greens', title='green band')
show((src, 3), ax=axr, cmap='Reds', title='red band')
show((src, 4), ax=axn, cmap='Greys', title='nir band')

In [ ]:
with rio.open("data/PlanetScope/PS_2019-08-29_mosaic.tif") as src:
    img_array = src.read()
# Ensure the input array doesn't have nodata values like -9999
# Use stretch = True
ep.plot_rgb(img_array,rgb=(2, 1, 0),stretch=True, title = "PS_2019-08-29_mosaic - RGB")

Water classification using band-ratio (Normalized Difference Water Index) combined with Otsu Thresholding.

The Normalized Difference Water Index (NDWI) is defined as follows:

**NDWI = (GREEN - NIR)/ (GREEN + NIR)**

Values closer to 1 represent water pixels, and values closer -1 represent non-water pixels

In [ ]:
# Open the raster file in read mode
PS_final = rio.open("data/PlanetScope/PS_2019-08-29_mosaic.tif")

# Metadata
PS_meta = PS_final.profile

# Read green and convert to float
PS_g = PS_final.read(2).astype('f4')

# Read NIR and convert to float
PS_nir = PS_final.read(4).astype('f4')

#calculate NDWI
PS_ndwi = (PS_g - PS_nir) / (PS_g + PS_nir)


In [ ]:
# let's plot using EarthPy functions
ep.hist(PS_ndwi,colors=["grey"],figsize=(6, 4), title="PS NDWI distribution", bins = 40)
ep.plot_bands(PS_ndwi, 
              cmap='YlGnBu',
              scale=False,
              vmin=np.nanmin(PS_ndwi), vmax=0.25, # I set vnax after checking the histogram;
              title="PS NDWI",figsize=(5, 5))
plt.show()